# Teltonika Data Sending Parameter ID Parser

## Panagiotis Kalogeropoulos, OpenRemote
## panos.kalogeropoulos@openremote.io

This Jupyter Notebook helps parse the Teltonika-GPS Wiki page for the data sending parameter IDs into a Pandas dataframe, which can then be exported into various types, like Excel or JSON. 

By parsing the table into programmatically accessible data, we are able to programmatically determine what parameter each Teltonika device is talking about. In this way, we can easily generalize to the entire line of Teltonika GPS Trackers.

In OpenRemote, this script will be used to automatically map parameters sent over by any Teltonika device into the correct attribute.


# Import required packages

Some packages are not being used directly, but they are required by other packages to run the application with no issue.



In [ ]:
import pandas
import pyarrow
import lxml
from re import sub

# Set Variables

The `device` variable is where the user would enter their device number, to retrieve the correct data from the Teltonika website.

In [ ]:
device = input("Enter Teltonika Telematics Model Number")
url="https://wiki.teltonika-gps.com/view/"+device+"_Teltonika_Data_Sending_Parameters_ID"
print(url)

# Download HTML of URL and parse into DataFrames

In [ ]:
dfList = pandas.read_html(url, header=1, encoding='utf-8')

⚠️Warning!⚠️

If you get an error relating to HTTP requests, you can change the `url` variable to point to the HTML file you manually downloaded from the website. This may happen to potential flagging of your IP address, as the Teltonika website uses Cloudflare to protect against abusive HTTP requests.

# Adjust DataFrames to export

Specifically, the page contains multiple tables, for example "Permanent I/O Elements", "Eventual I/O elements", etc.

Thus, by combining the tables, we gather a singular table with the Parameter Group column guaranteeing no data loss from the concatenation.

# DataFrame adjustments

Converting headers to camelCase

In [ ]:
def camel_case(s):
    s = sub(r"(_|-)+", " ", s).title().replace(" ", "")
    return ''.join([s[0].lower(), s[1:]])

In [ ]:
df_copy = dfList.copy()

for df in df_copy:
    df.rename(columns=camel_case, inplace=True)

In [ ]:
# These properties are the only ones we ACTUALLY require. 
# The rest can be a "-", and we can handle that. 
# As long as we have those, we're set. 
# Worst case scenario, the values are parsed as strings,
# And someone goes through those SPECIFIC properties,
# Adds the types, and proceeds. People can do a find and replace.
required_columns = {"propertyIdInAvlPacket", "propertyName", "type"}
main_columns = set(df_copy[0].columns)


for idx, df in enumerate(df_copy):
    if required_columns.issubset(df.columns):
        # Drop all elements that are in df.columns but not main_columns
        df.drop(list(df.columns.difference(main_columns)), axis=1, inplace=True)
        #find the columns that exist in main_columns and not in df.columns
        missing = main_columns.difference(df.columns)
        # Fill in the value as "-", as Teltonika does
        for col in missing:
            df[col] = "-"
    # if the selected `df` does not contain the basic required_columns, just drop it from the list.
    # We cannot cover every single edge case. The best thing we can do is let the user know.
    else: 
        print("DATAFRAME CORRUPTED - CANNOT PARSE:")
        print(df)
        df_copy.pop(idx)

megaDf = pandas.concat(df_copy, ignore_index=True)
    

In [ ]:
# Drop the index that was automatically inserted
megaDf.reset_index(drop=True, inplace=True)
megaDf

# Export to files

Look into the pandas documentation for other export types.

In [ ]:
megaDf.to_excel(device+".xlsx")
# Dealing with Unicode errors
with open(device+".json", 'w', encoding='utf-8') as file:
    megaDf.to_json(file, force_ascii=False, orient="records", indent=4)

# Show data types

Using the below command, we can see the unique data types in the exported file, that will be used by OpenRemote to determine the type of the received value.

In [ ]:
print(megaDf['type'].unique())

# Check the file!

Go ahead and check the file. Use various tools to plot the file into a table, or export it as an excel file, to see the exported results. Fix any issues, and then import it into OpenRemote. 
